In [1]:
import pandas as pd
import ast
import json
import base64
import numpy as np
import os
import cv2, random
np.set_printoptions(suppress=True, precision=4)
from scipy import spatial
import numpy as np
import yaml
import os.path as op
from tqdm import tqdm
import nltk
from nltk.corpus import wordnet
tqdm.pandas()

Detection

In [2]:
sg_tsv = './features_detection/training/train.feature.tsv'
df_feature_detection = pd.read_csv(sg_tsv,sep='\t',header = None, converters={1:json.loads}, skiprows = 20000, nrows = 10000)
df_feature_detection = df_feature_detection.rename(columns={0: "id", 1: "features"})

In [3]:
sg_tsv = './features_detection/training/train.label.tsv'
df_label_detection = pd.read_csv(sg_tsv,sep='\t',header = None, converters={1:json.loads}, skiprows = 20000, nrows=10000)
df_label_detection=df_label_detection.rename(columns={0: "id", 1: "label"})
#df_label_detection

In [4]:
full_detection = pd.merge(df_feature_detection, df_label_detection, on="id")
#full_detection

In [5]:
from tqdm import tqdm
bbox_id = 0
for index, row in tqdm(full_detection.iterrows()):
    list_features = np.split(np.frombuffer(base64.b64decode(row["features"]["features"]), np.float32), row["features"]["num_boxes"])
    for i in range(row["features"]["num_boxes"]):
        row["label"][i]["feature"] = list_features[i]
        row["label"][i]["bbox_id"] = bbox_id
        bbox_id+=1

9000it [01:00, 149.79it/s]


Segmentation

In [6]:
sg_tsv = './features_test1/training/train.feature.tsv'
df_feature_segmentation = pd.read_csv(sg_tsv,sep='\t',header = None, converters={1:json.loads}, skiprows = 20000, nrows=10000)
df_feature_segmentation = df_feature_segmentation.rename(columns={0: "id", 1: "features"})

In [7]:
sg_tsv = './features_test1/training/train.label.tsv'
df_label_segmentation = pd.read_csv(sg_tsv,sep='\t',header = None, converters={1:json.loads}, skiprows = 20000, nrows=10000)
df_label_segmentation = df_label_segmentation.rename(columns={0: "id", 1: "label"})
#df_label_segmentation

In [8]:
full_segmentation = pd.merge(df_feature_segmentation, df_label_segmentation, on="id")
full_segmentation

,id,features,label
0,4679648220,{'features': 'V9OhP+JB4UCWO41Ban4SQRZjtEAAAAAA...,"[{'class': 'Bicycle', 'conf': 0.97119373083114..."
1,4679736464,{'features': '4jMaQbnrkEHZc3JAmLC7P0mkbkEAAAAA...,"[{'class': 'Life jacket', 'conf': 0.9988144636..."
2,4679761535,{'features': 'wA33QL8ifkHEDVRAQXEOQRc/UkB0TSFA...,"[{'class': 'Dress', 'conf': 0.975189208984375,..."
3,4679771579,{'features': 'A3zNQMSSyEBsZfU+n5+yQChyskBOZgw/...,"[{'class': 'Skirt', 'conf': 0.9394164085388184..."
4,4679881654,{'features': 'NvofQf5EQUBYbn9B3oalQIritD8Naes/...,"[{'class': 'Flag', 'conf': 0.8809290528297424,..."
...,...,...,...
8995,997338199,{'features': 'aPgEQQAAAAAAAAAAJX5TQAAAAADvJSY+...,"[{'class': 'Curtain', 'conf': 0.80887413024902..."
8996,997722733,{'features': '9pzPP7/XVUEAAAAAnGWUPuE1T0H3ks1A...,"[{'class': 'Sandal', 'conf': 0.717478156089782..."
8997,997876722,{'features': 'mPJwQAAAAAAAAAAAuo71PzYZgkAaSe5A...,"[{'class': 'Apple', 'conf': 0.9743674397468567..."
8998,99804383,{'features': 'SwyYQBGga0A6YDY/AAAAAJCAgj8fi/s/...,"[{'class': 'Signboard', 'conf': 0.706024229526..."


In [9]:
from tqdm import tqdm
bbox_id = 0
for index, row in tqdm(full_segmentation.iterrows()):
    list_features = np.split(np.frombuffer(base64.b64decode(row["features"]["features"]), np.float32), row["features"]["num_boxes"])
    for i in range(row["features"]["num_boxes"]):
        row["label"][i]["feature"] = list_features[i]
        row["label"][i]["bbox_id"] = bbox_id
        bbox_id+=1

9000it [00:20, 449.28it/s]


Union

In [10]:
full_segmentation["union"] = full_segmentation["label"] + full_detection["label"]
full_segmentation 

,id,features,label,union
0,4679648220,{'features': 'V9OhP+JB4UCWO41Ban4SQRZjtEAAAAAA...,"[{'class': 'Bicycle', 'conf': 0.97119373083114...","[{'class': 'Bicycle', 'conf': 0.97119373083114..."
1,4679736464,{'features': '4jMaQbnrkEHZc3JAmLC7P0mkbkEAAAAA...,"[{'class': 'Life jacket', 'conf': 0.9988144636...","[{'class': 'Life jacket', 'conf': 0.9988144636..."
2,4679761535,{'features': 'wA33QL8ifkHEDVRAQXEOQRc/UkB0TSFA...,"[{'class': 'Dress', 'conf': 0.975189208984375,...","[{'class': 'Dress', 'conf': 0.975189208984375,..."
3,4679771579,{'features': 'A3zNQMSSyEBsZfU+n5+yQChyskBOZgw/...,"[{'class': 'Skirt', 'conf': 0.9394164085388184...","[{'class': 'Skirt', 'conf': 0.9394164085388184..."
4,4679881654,{'features': 'NvofQf5EQUBYbn9B3oalQIritD8Naes/...,"[{'class': 'Flag', 'conf': 0.8809290528297424,...","[{'class': 'Flag', 'conf': 0.8809290528297424,..."
...,...,...,...,...
8995,997338199,{'features': 'aPgEQQAAAAAAAAAAJX5TQAAAAADvJSY+...,"[{'class': 'Curtain', 'conf': 0.80887413024902...","[{'class': 'Curtain', 'conf': 0.80887413024902..."
8996,997722733,{'features': '9pzPP7/XVUEAAAAAnGWUPuE1T0H3ks1A...,"[{'class': 'Sandal', 'conf': 0.717478156089782...","[{'class': 'Sandal', 'conf': 0.717478156089782..."
8997,997876722,{'features': 'mPJwQAAAAAAAAAAAuo71PzYZgkAaSe5A...,"[{'class': 'Apple', 'conf': 0.9743674397468567...","[{'class': 'Apple', 'conf': 0.9743674397468567..."
8998,99804383,{'features': 'SwyYQBGga0A6YDY/AAAAAJCAgj8fi/s/...,"[{'class': 'Signboard', 'conf': 0.706024229526...","[{'class': 'Signboard', 'conf': 0.706024229526..."


In [11]:
for i in tqdm(range(len(full_segmentation))):
    if(len(full_segmentation["label"].iloc[i]) + len(full_detection["label"].iloc[i]) != len(full_segmentation["union"].iloc[i])):
        print("!!!!!!!!!!!!!!!")

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9000/9000 [00:00<00:00, 19396.54it/s]


In [12]:
def generate_features(x):
    idx, data, num_boxes = x["id"],x["union"],len(x["union"])
    features_arr = []
    for i in range(num_boxes):
        features = data[i]['feature']
        features_arr.append(features.astype(np.float32))
    
    features = np.vstack(tuple(features_arr))
    features = base64.b64encode(features).decode("utf-8")
    return {"features":features, "num_boxes":num_boxes}

In [13]:
def generate_labels(x):
    data = x["union"]
    res = [{"class":el['class'].capitalize(),"conf":el['conf'], "rect": el['rect']} for el in data] 
    return res

In [14]:
full_segmentation['feature_union'] = full_segmentation.progress_apply(generate_features,axis=1)
full_segmentation['feature_union'] = full_segmentation['feature_union'].progress_apply(json.dumps)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9000/9000 [01:27<00:00, 102.47it/s]


In [15]:
full_segmentation['label_union'] = full_segmentation.progress_apply(generate_labels,axis=1)
full_segmentation['label_union'] = full_segmentation['label_union'].progress_apply(json.dumps)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9000/9000 [00:04<00:00, 1875.00it/s]


In [16]:
from features_detection.maskrcnn_benchmark.structures.tsv_file_ops import tsv_reader, tsv_writer

OUTPUT_DIR = './training_union_full/'
LABEL_FILE = os.path.join(OUTPUT_DIR,'train3.label.tsv')
FEATURE_FILE = os.path.join(OUTPUT_DIR,'train3.feature.tsv')
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)
    print(f"path to {OUTPUT_DIR} created")

In [17]:
tsv_writer(full_segmentation[["id", "feature_union"]].values.tolist(), FEATURE_FILE)

In [18]:
tsv_writer(full_segmentation[["id", "label_union"]].values.tolist(), LABEL_FILE)

Read

In [2]:
sg_tsv = './training_union_full/train1.feature.tsv'
df_features_1 = pd.read_csv(sg_tsv,sep='\t',header = None, converters={1:json.loads})
df_features_1 = df_features_1.rename(columns={0: "id", 1: "feature"})
df_features_1

,id,label
0,1000092795,{'features': 'ki89QAAAAAAAAAAAFvs8P3NUhz/ght1A...
1,10002456,{'features': 'Ta8CQRDNxD81+ZRAPhfGQOb8wkCgvb5A...
2,1000268201,{'features': 'fKqDQQAAAAA8P44/WHkaQfywkEAAAAAA...
3,1000344755,{'features': 'u24FQFOHokAAAAAACkI9QBn70T8kwudA...
4,1000366164,{'features': 'lJYiQV6dlT9WZDlAXJmOQL94t0AAshc8...
...,...,...
9995,307994435,{'features': 'AGhLPKznuz4AAAAAhVh6QHDGqT2QM/M9...
9996,3080056515,{'features': 'eGP2QPKWuEEAAAAAwxWbQI4NAEAAAAAA...
9997,308014594,{'features': 'hBGQQGaBuUD8eHRAwoQeQYS/bD8AAAAA...
9998,3080363242,{'features': 'SOWQQN9P/j/Bx7ZA+SWRQQRBpkAAAAAA...


In [24]:
sg_tsv = './training_union_full/train1.label.tsv'
df_labels_1 = pd.read_csv(sg_tsv,sep='\t',header = None, converters={1:json.loads})
df_labels_1 = df_labels_1.rename(columns={0: "id", 1: "label"})
df_labels_1

,id,label
0,1000092795,"[{'class': 'Belt buckle', 'conf': 0.5350186228..."
1,10002456,"[{'class': 'Traffic light', 'conf': 0.51291126..."
2,1000268201,"[{'class': 'Dress', 'conf': 0.9662193059921265..."
3,1000344755,"[{'class': 'Rubber band', 'conf': 0.7075430750..."
4,1000366164,"[{'class': 'Drum (musical instrument)', 'conf'..."
...,...,...
9995,307994435,"[{'class': 'Signboard', 'conf': 0.608466207981..."
9996,3080056515,"[{'class': 'Scarf', 'conf': 0.6872310638427734..."
9997,308014594,"[{'class': 'Hat', 'conf': 0.734725296497345, '..."
9998,3080363242,"[{'class': 'Jean', 'conf': 0.6286819577217102,..."


In [3]:
sg_tsv = './training_union_full/train2.feature.tsv'
df_features_2 = pd.read_csv(sg_tsv,sep='\t',header = None, converters={1:json.loads})
df_features_2 = df_features_2.rename(columns={0: "id", 1: "feature"})
df_features_2

,id,label
0,3081182021,{'features': 'O6M/QMDEvD0AAAAAAAAAAAAAAACmFdtA...
1,3081330705,{'features': 'AAAAAJHuu0AAAAAAgBkCQUh82j+y1nE/...
2,30813349,{'features': '5S1oQKQABT8d/AdBJz3+QMzuGUAAAAAA...
3,3081363964,{'features': 'fBLBQLJQykGg56s90OxBQO4bYEDCIIs/...
4,3081395629,{'features': 'AAAAAAD9DD3ULytBss0UQd0JgUEeel4/...
...,...,...
9995,4679327842,{'features': 'OBFbQAAAAACWi6xAoCdiQB9Qlz/yNz5A...
9996,4679363874,{'features': 'AAAAACrkDz++UhlB+Gk8QVQz6UDPdENA...
9997,4679525981,{'features': 'fDOtQKi6h0Gguu5AGvhBQAAAAADDATpA...
9998,467960888,{'features': '7aLrQOXtQUCiQINAZQncPwAAAACy3N5A...


In [22]:
sg_tsv = './training_union_full/train2.label.tsv'
df_labels_2 = pd.read_csv(sg_tsv,sep='\t',header = None, converters={1:json.loads})
df_labels_2 = df_labels_2.rename(columns={0: "id", 1: "label"})
df_labels_2

,id,label
0,3081182021,"[{'class': 'Light bulb', 'conf': 0.90654027462..."
1,3081330705,"[{'class': 'Sock', 'conf': 0.6373504400253296,..."
2,30813349,"[{'class': 'Manhole', 'conf': 0.93582820892333..."
3,3081363964,"[{'class': 'Grass', 'conf': 0.9957932233810425..."
4,3081395629,"[{'class': 'Bracelet', 'conf': 0.7277402877807..."
...,...,...
9995,4679327842,"[{'class': 'Frisbee', 'conf': 0.57207930088043..."
9996,4679363874,"[{'class': 'Backpack', 'conf': 0.9342467188835..."
9997,4679525981,"[{'class': 'Street sign', 'conf': 0.8102518320..."
9998,467960888,"[{'class': 'Sea', 'conf': 0.9997655749320984, ..."


In [4]:
sg_tsv = './training_union_full/train3.feature.tsv'
df_features_3 = pd.read_csv(sg_tsv,sep='\t',header = None, converters={1:json.loads})
df_features_3 = df_features_3.rename(columns={0: "id", 1: "feature"})
df_features_3

,id,label
0,4679648220,{'features': 'V9OhP+JB4UCWO41Ban4SQRZjtEAAAAAA...
1,4679736464,{'features': '4jMaQbnrkEHZc3JAmLC7P0mkbkEAAAAA...
2,4679761535,{'features': 'wA33QL8ifkHEDVRAQXEOQRc/UkB0TSFA...
3,4679771579,{'features': 'A3zNQMSSyEBsZfU+n5+yQChyskBOZgw/...
4,4679881654,{'features': 'NvofQf5EQUBYbn9B3oalQIritD8Naes/...
...,...,...
8995,997338199,{'features': 'aPgEQQAAAAAAAAAAJX5TQAAAAADvJSY+...
8996,997722733,{'features': '9pzPP7/XVUEAAAAAnGWUPuE1T0H3ks1A...
8997,997876722,{'features': 'mPJwQAAAAAAAAAAAuo71PzYZgkAaSe5A...
8998,99804383,{'features': 'SwyYQBGga0A6YDY/AAAAAJCAgj8fi/s/...


In [23]:
sg_tsv = './training_union_full/train3.label.tsv'
df_labels_3 = pd.read_csv(sg_tsv,sep='\t',header = None, converters={1:json.loads})
df_labels_3 = df_labels_3.rename(columns={0: "id", 1: "label"})
df_labels_3

,id,label
0,4679648220,"[{'class': 'Bicycle', 'conf': 0.97119373083114..."
1,4679736464,"[{'class': 'Life jacket', 'conf': 0.9988144636..."
2,4679761535,"[{'class': 'Dress', 'conf': 0.975189208984375,..."
3,4679771579,"[{'class': 'Skirt', 'conf': 0.9394164085388184..."
4,4679881654,"[{'class': 'Flag', 'conf': 0.8809290528297424,..."
...,...,...
8995,997338199,"[{'class': 'Curtain', 'conf': 0.80887413024902..."
8996,997722733,"[{'class': 'Sandal', 'conf': 0.717478156089782..."
8997,997876722,"[{'class': 'Apple', 'conf': 0.9743674397468567..."
8998,99804383,"[{'class': 'Signboard', 'conf': 0.706024229526..."


In [7]:
df_features = df_features_1.append([df_features_2, df_features_3], ignore_index=True)

df_features

,id,feature
0,1000092795,{'features': 'ki89QAAAAAAAAAAAFvs8P3NUhz/ght1A...
1,10002456,{'features': 'Ta8CQRDNxD81+ZRAPhfGQOb8wkCgvb5A...
2,1000268201,{'features': 'fKqDQQAAAAA8P44/WHkaQfywkEAAAAAA...
3,1000344755,{'features': 'u24FQFOHokAAAAAACkI9QBn70T8kwudA...
4,1000366164,{'features': 'lJYiQV6dlT9WZDlAXJmOQL94t0AAshc8...
...,...,...
28995,997338199,{'features': 'aPgEQQAAAAAAAAAAJX5TQAAAAADvJSY+...
28996,997722733,{'features': '9pzPP7/XVUEAAAAAnGWUPuE1T0H3ks1A...
28997,997876722,{'features': 'mPJwQAAAAAAAAAAAuo71PzYZgkAaSe5A...
28998,99804383,{'features': 'SwyYQBGga0A6YDY/AAAAAJCAgj8fi/s/...


In [25]:
df_labels = df_labels_1.append([df_labels_2, df_labels_3], ignore_index=True)
df_labels

,id,label
0,1000092795,"[{'class': 'Belt buckle', 'conf': 0.5350186228..."
1,10002456,"[{'class': 'Traffic light', 'conf': 0.51291126..."
2,1000268201,"[{'class': 'Dress', 'conf': 0.9662193059921265..."
3,1000344755,"[{'class': 'Rubber band', 'conf': 0.7075430750..."
4,1000366164,"[{'class': 'Drum (musical instrument)', 'conf'..."
...,...,...
28995,997338199,"[{'class': 'Curtain', 'conf': 0.80887413024902..."
28996,997722733,"[{'class': 'Sandal', 'conf': 0.717478156089782..."
28997,997876722,"[{'class': 'Apple', 'conf': 0.9743674397468567..."
28998,99804383,"[{'class': 'Signboard', 'conf': 0.706024229526..."


In [8]:
sg_tsv = './training_union_full/train.hw.tsv'

df_train_hw = pd.read_csv(sg_tsv,sep='\t',header = None,converters={1:json.loads})#converters={1:ast.literal_eval})
df_train_hw = df_train_hw.rename(columns={0: "id", 1: "dim"})

df_train_hw

,id,dim
0,1000092795,"[{'height': 500, 'width': 333}]"
1,10002456,"[{'height': 374, 'width': 500}]"
2,1000268201,"[{'height': 500, 'width': 375}]"
3,1000344755,"[{'height': 333, 'width': 500}]"
4,1000366164,"[{'height': 375, 'width': 500}]"
...,...,...
28995,997338199,"[{'height': 400, 'width': 500}]"
28996,997722733,"[{'height': 500, 'width': 333}]"
28997,997876722,"[{'height': 500, 'width': 333}]"
28998,99804383,"[{'height': 500, 'width': 375}]"


In [9]:
df_features = df_train_hw.merge(df_features, on=["id"])[["id","feature"]]
df_features

,id,feature
0,1000092795,{'features': 'ki89QAAAAAAAAAAAFvs8P3NUhz/ght1A...
1,10002456,{'features': 'Ta8CQRDNxD81+ZRAPhfGQOb8wkCgvb5A...
2,1000268201,{'features': 'fKqDQQAAAAA8P44/WHkaQfywkEAAAAAA...
3,1000344755,{'features': 'u24FQFOHokAAAAAACkI9QBn70T8kwudA...
4,1000366164,{'features': 'lJYiQV6dlT9WZDlAXJmOQL94t0AAshc8...
...,...,...
28995,997338199,{'features': 'aPgEQQAAAAAAAAAAJX5TQAAAAADvJSY+...
28996,997722733,{'features': '9pzPP7/XVUEAAAAAnGWUPuE1T0H3ks1A...
28997,997876722,{'features': 'mPJwQAAAAAAAAAAAuo71PzYZgkAaSe5A...
28998,99804383,{'features': 'SwyYQBGga0A6YDY/AAAAAJCAgj8fi/s/...


In [31]:
df_labels = df_train_hw.merge(df_labels, on=["id"])[["id","label"]]
df_labels

,id,label
0,1000092795,"[{'class': 'Belt buckle', 'conf': 0.5350186228..."
1,10002456,"[{'class': 'Traffic light', 'conf': 0.51291126..."
2,1000268201,"[{'class': 'Dress', 'conf': 0.9662193059921265..."
3,1000344755,"[{'class': 'Rubber band', 'conf': 0.7075430750..."
4,1000366164,"[{'class': 'Drum (musical instrument)', 'conf'..."
...,...,...
28995,997338199,"[{'class': 'Curtain', 'conf': 0.80887413024902..."
28996,997722733,"[{'class': 'Sandal', 'conf': 0.717478156089782..."
28997,997876722,"[{'class': 'Apple', 'conf': 0.9743674397468567..."
28998,99804383,"[{'class': 'Signboard', 'conf': 0.706024229526..."


In [13]:
df_features['feature'] = df_features['feature'].progress_apply(json.dumps)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 29000/29000 [09:51<00:00, 48.99it/s]


In [14]:
df_features

,id,feature
0,1000092795,"{""features"": ""ki89QAAAAAAAAAAAFvs8P3NUhz/ght1A..."
1,10002456,"{""features"": ""Ta8CQRDNxD81+ZRAPhfGQOb8wkCgvb5A..."
2,1000268201,"{""features"": ""fKqDQQAAAAA8P44/WHkaQfywkEAAAAAA..."
3,1000344755,"{""features"": ""u24FQFOHokAAAAAACkI9QBn70T8kwudA..."
4,1000366164,"{""features"": ""lJYiQV6dlT9WZDlAXJmOQL94t0AAshc8..."
...,...,...
28995,997338199,"{""features"": ""aPgEQQAAAAAAAAAAJX5TQAAAAADvJSY+..."
28996,997722733,"{""features"": ""9pzPP7/XVUEAAAAAnGWUPuE1T0H3ks1A..."
28997,997876722,"{""features"": ""mPJwQAAAAAAAAAAAuo71PzYZgkAaSe5A..."
28998,99804383,"{""features"": ""SwyYQBGga0A6YDY/AAAAAJCAgj8fi/s/..."


In [17]:
OUTPUT_DIR = './training_union_full/'

FEATURE_FILE = os.path.join(OUTPUT_DIR,'train.feature.tsv')

tsv_writer(df_features[["id",'feature']].values.tolist(),FEATURE_FILE)

In [32]:
df_labels['label'] = df_labels['label'].progress_apply(json.dumps)

In [33]:
OUTPUT_DIR = './training_union_full/'

LABEL_FILE = os.path.join(OUTPUT_DIR,'train.label.tsv')

tsv_writer(df_labels[["id",'label']].values.tolist(), LABEL_FILE)

In [ ]:
# test

In [3]:
sg_tsv = './training_union_full/train.feature.tsv'
df_features = pd.read_csv(sg_tsv,sep='\t',header = None, converters={1:json.loads}, skiprows = 16000, nrows = 10000)
df_features = df_features.rename(columns={0: "id", 1: "feature"})
df_features

,id,feature
0,3947306345,{'features': '6jDTQCktF0EAAAAAuJKPQDm8DkG7V9RA...
1,3947686324,{'features': 'lII3QUhAvEAAAAAA/udsQKUnD0CAAHk9...
2,3947729724,{'features': 'lWiEQbIzEUEsxQtAqQG0QJJAs0Ao1hZA...
3,3947925542,{'features': 'dJaLP3qOM0Dy/+w/6hBPQQ7QU0BUns8/...
4,3948611540,{'features': 'IVIaQVkFsUAAAAAAv9g7QEbihUAAAAAA...
...,...,...
9995,6314474316,{'features': 'POULQQAAAAAAAAAA54vTPhs8bUB6v5U/...
9996,6314475602,{'features': 'YvejQJa54T9ItsA+7PBKPynirEAAAAAA...
9997,6314475788,{'features': '0Ca8QAAAAAAAAAAA6ZvFQGZOHz3GfBlA...
9998,6315045481,{'features': 'v5ELQXs2aEEAAAAAhBTNP3AQu0Duhbk/...


In [5]:
sg_tsv = './training_union_full/train.label.tsv'
df_labels = pd.read_csv(sg_tsv,sep='\t',header = None, converters={1:json.loads})
df_labels = df_labels.rename(columns={0: "id", 1: "label"})
df_labels

,id,label
0,1000092795,"[{'class': 'Belt buckle', 'conf': 0.5350186228..."
1,10002456,"[{'class': 'Traffic light', 'conf': 0.51291126..."
2,1000268201,"[{'class': 'Dress', 'conf': 0.9662193059921265..."
3,1000344755,"[{'class': 'Rubber band', 'conf': 0.7075430750..."
4,1000366164,"[{'class': 'Drum (musical instrument)', 'conf'..."
...,...,...
28995,997338199,"[{'class': 'Curtain', 'conf': 0.80887413024902..."
28996,997722733,"[{'class': 'Sandal', 'conf': 0.717478156089782..."
28997,997876722,"[{'class': 'Apple', 'conf': 0.9743674397468567..."
28998,99804383,"[{'class': 'Signboard', 'conf': 0.706024229526..."


In [4]:
for elem in tqdm(df_features["feature"].values): # check
    if len(np.frombuffer(base64.b64decode(elem["features"]), np.float32)) / elem["num_boxes"] != 2054:
        print("noooooooooooooooo!!!!!!!")

100%|███████████████████████████████████████████████████████████████████████████| 10000/10000 [01:00<00:00, 166.08it/s]


In [14]:
#df_labels.merge(df_features, on=["id"])["feature"].iloc[0]
for elem, label in tqdm(zip(df_labels.merge(df_features, on=["id"])["feature"].values, df_labels.merge(df_features, on=["id"])["label"].values)): # check
    if elem["num_boxes"] != len(label):
        print("noooooooooooooooo!!!!!!!")#num_boxes

10000it [00:00, 1242830.39it/s]


In [17]:
df_labels.merge(df_features, on=["id"])["feature"].iloc[0]["num_boxes"] == len(df_labels.merge(df_features, on=["id"])["label"].iloc[0])

True